In [1]:
# import Library and some random image data set
import tensorflow as tf
import numpy      as np
import seaborn    as sns 
import pandas     as pd
import os
import matplotlib.pyplot as plt
np.random.seed(78); tf.set_random_seed(78)

# get some of the STL data set
from skimage import util 
from skimage.transform import resize
from skimage.io import imread
import warnings
warnings.filterwarnings("ignore")

In [11]:
# create the layers

def tf_elu(x):   return tf.nn.elu(x)
def d_tf_elu(x): return tf.cast(tf.greater(x,0),tf.float64)  + (tf_elu(tf.cast(tf.less_equal(x,0),tf.float64) * x) + 1.0)

def tf_relu(x):   return tf.nn.relu(x)
def d_tf_relu(x): return tf.cast(tf.greater(x,0),tf.float64)

def tf_tanh(x):   return tf.nn.tanh(x)
def d_tf_tanh(x): return 1 - tf_tanh(x) ** 2

def tf_sigmoid(x):   return tf.nn.sigmoid(x)
def d_tf_sigmoid(x): return tf_sigmoid(x) * (1.0-tf_sigmoid(x))

class FNN():
    
    def __init__(self,inc,outc,act,dact):
        self.w = tf.Variable(tf.random_normal((inc,outc),stddev=0.01))
        self.act = act 
        self.dact= dact
        
    def feedforward(self,data):
        self.input = data
        self.layer = data @ self.w
        self.layera= self.act(self.layer)
        return [self.layer,self.layera]
    
    def backprop(self,grad):
        grad1 = grad
        grad2 = self.dact(self.layer)
        grad3 = self.input
        grad_middle = grad1 * grad2
        
        gradw = tf.transpose(self.input) @ grad_middle
        gradp = grad_middle @ tf.transpose(self.w)
        
        return [gradw,gradp]
    
class tf_batch_norm_layer():
    
    def __init__(self,axis):
        self.axis        = axis
        
    def feedforward(self,input,training_phase=True,eps = 1e-8):
        self.input = input
        self.input_size          = self.input.shape
        self.batch,self.c = self.input_size[0].value,self.input_size[1].value

        # Training Moving Average Mean         
        def training_fn():
            self.mean    = tf.reduce_mean(self.input,axis=self.axis ,keepdims=True)
            self.var     = tf.reduce_mean(tf.square(self.input-self.mean),axis=self.axis,keepdims=True)
            centered_data= (self.input - self.mean)/tf.sqrt(self.var + eps)
            return centered_data
        
        # Testing Moving Average Mean        
        def  testing_fn():
            centered_data   = (self.input - self.moving_mean)/tf.sqrt(self.moving_vari + eps)
            return centered_data
        
        self.output = tf.cond(training_phase,true_fn=training_fn,false_fn=testing_fn)
        return self.output
    
    def backprop(self,grad,eps = 1e-8):
        change_parts = 1.0 /(self.batch * self.h * self.w)
        grad_sigma   = tf.reduce_sum( grad *  (self.input-self.mean)     ,axis=self.axis,keepdims=True) * -0.5 * (self.var+eps) ** -1.5
        grad_mean    = tf.reduce_sum( grad *  (-1./tf.sqrt(self.var+eps)),axis=self.axis,keepdims=True) + grad_sigma * change_parts * 2.0 * tf.reduce_sum((self.input-self.mean),axis=self.axis,keepdims=True) * -1
        grad_x       = grad * 1/(tf.sqrt(self.var+eps)) + grad_sigma * change_parts * 2.0 * (self.input-self.mean) + grad_mean * change_parts
        return grad_x

In [12]:
# declare layers
l1 = FNN(100,64,tf_relu,d_tf_relu)
l1b= tf_batch_norm_layer(0)
l2 = FNN(64,32,tf_relu,d_tf_relu)
l2b= tf_batch_norm_layer(0)
l3 = FNN(32,16,tf_relu,d_tf_relu)
l3b= tf_batch_norm_layer(0)
l4 = FNN(16,1,tf_relu,d_tf_relu)

In [13]:
# create some random data
x_data = np.random.uniform(0,1,(20,100))

layer1 = l1.feedforward(x_data)
layer2 = l2.feedforward(layer1)
layer3 = l3.feedforward(layer2)
layer4 = l4.feedforward(layer3)

print(layer1)
print(layer1)
print(layer1)
print(layer1)

